# LightGBM

In [17]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from numpy.random import seed

# 随机种子
seed(1111)

# 读取CSV文件，指定"gene"列作为索引
label = pd.read_csv('~/wuzhenao/divide/seq_fish_find_var.csv', index_col='gene')['label']

# 读取数据，排除"gene"和"label"列
data = pd.read_csv('~/wuzhenao/divide/seq_fish_find_var.csv').drop(columns=['gene', 'label'])

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.25)

# 转换为Dataset数据格式
train_data = lgb.Dataset(X_train, label=y_train)
validation_data = lgb.Dataset(X_test, label=y_test)

# 参数
params = {
    'learning_rate': 0.1,
    'lambda_l1': 0.1,
    'lambda_l2': 0,
    'max_depth': 4,
    'objective': 'multiclass',  # 目标函数
    'num_class': 2,
}

# 模型训练
gbm = lgb.train(params, train_data, valid_sets=[validation_data])

# 模型预测
y_pred = gbm.predict(X_test)
y_pred = [list(x).index(max(x)) for x in y_pred]
print(y_pred)

# 模型评估
print(accuracy_score(y_test, y_pred))


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.091334 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 255
[LightGBM] [Info] Number of data points in the train set: 1500, number of used features: 1
[LightGBM] [Info] Start training from score -0.193585
[LightGBM] [Info] Start training from score -1.737271
[1]	valid_0's multi_logloss: 0.470558
[2]	valid_0's multi_logloss: 0.469753
[3]	valid_0's multi_logloss: 0.469948
[4]	valid_0's multi_logloss: 0.469491
[5]	valid_0's multi_logloss: 0.470939
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[6]	valid_0's multi_logloss: 0.471193
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[7]	valid_0's multi_logloss: 0.471694
[LightGBM] [Warning] No further split

# Random Forest

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from numpy.random import seed

# 读取CSV文件，指定"gene"列作为索引
data = pd.read_csv('~/wuzhenao/divide/seq_fish_find_var.csv')

# 使用LabelEncoder对基因列进行编码
gene_encoder = LabelEncoder()
data['gene'] = gene_encoder.fit_transform(data['gene'])

# 将标签列的数据类型转换为整数
data['label'] = data['label'].astype(int)

# 划分训练集和测试集
Xtrain, Xtest, Ytrain, Ytest = train_test_split(data.drop(columns=['label']), data['label'], test_size=0.25, random_state=1)

# 将Pandas DataFrame转换为NumPy数组
Xtrain = np.array(Xtrain)
Ytrain = np.array(Ytrain)
Xtest = np.array(Xtest)
Ytest = np.array(Ytest)

# 建立模型
clf = DecisionTreeClassifier(random_state=1, max_depth=5)  # 限制决策树的深度
rfc = RandomForestClassifier(random_state=1, n_estimators=100)  # 增加随机森林的树的数量

clf = clf.fit(Xtrain, Ytrain)
rfc = rfc.fit(Xtrain, Ytrain)

# 查看模型效果
score_c = clf.score(Xtest, Ytest)
score_r = rfc.score(Xtest, Ytest)

# 打印最后结果
print("Single Tree:", score_c)
print("Random Forest:", score_r)


Single Tree: 0.832
Random Forest: 0.79


# CPPLS-MLP

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


# 读取CSV文件，指定"gene"列作为索引
data = pd.read_csv('~/wuzhenao/divide/seq_fish_find_var.csv')

# 将基因名称进行独热编码
gene_encoder = OneHotEncoder(sparse=False)
gene_encoded = gene_encoder.fit_transform(data[['gene']])

# 读取标签列
label = data['label']

# 划分训练集和测试集
Xtrain, Xtest, Ytrain, Ytest = train_test_split(gene_encoded, label, test_size=0.25, random_state=1)

# 特征标准化
scaler = StandardScaler()
Xtrain = scaler.fit_transform(Xtrain)
Xtest = scaler.transform(Xtest)

# 建立神经网络模型
model = keras.Sequential([
    layers.Input(shape=(Xtrain.shape[1],)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 训练模型
history = model.fit(Xtrain, Ytrain, epochs=100, batch_size=32, validation_data=(Xtest, Ytest))

# 评估模型
test_loss, test_acc = model.evaluate(Xtest, Ytest)
print("Test accuracy:", test_acc)


Epoch 1/100


/home/tjzhang01/miniconda3/envs/PLS/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


47/47 [==============================] - 1s 8ms/step - loss: 0.5454 - accuracy: 0.7840 - val_loss: 0.5838 - val_accuracy: 0.8460
Epoch 2/100
47/47 [==============================] - 0s 4ms/step - loss: 0.2487 - accuracy: 0.8880 - val_loss: 0.5443 - val_accuracy: 0.8460
Epoch 3/100
47/47 [==============================] - 0s 5ms/step - loss: 0.0769 - accuracy: 0.9953 - val_loss: 0.4876 - val_accuracy: 0.8460
Epoch 4/100
47/47 [==============================] - 0s 4ms/step - loss: 0.0163 - accuracy: 0.9993 - val_loss: 0.4801 - val_accuracy: 0.8460
Epoch 5/100
47/47 [==============================] - 0s 5ms/step - loss: 0.0055 - accuracy: 1.0000 - val_loss: 0.4484 - val_accuracy: 0.8460
Epoch 6/100
47/47 [==============================] - 0s 5ms/step - loss: 0.0029 - accuracy: 1.0000 - val_loss: 0.4408 - val_accuracy: 0.8460
Epoch 7/100
47/47 [==============================] - 0s 5ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.4346 - val_accuracy: 0.8460
Epoch 8/100
47/47 [======